Table 2a:  Number of alcohol related deaths by age and registration year, 2006-2016 - (new definition)

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Alcohol_Tables_16_0.xls'
inputFile = sourceFolder / 'Alcohol_Tables_16_0.xls'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(response.content)

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table 2a')[0]

Loading in\Alcohol_Tables_16_0.xls which has size 969216 bytes
Table names: ['Table 2a']


In [4]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()

In [5]:
observations

{<E13 28.0>, <E16 442.0>, <H5 22.0>, <H15 47.0>, <J8 53.0>, <G9 76.0>, <G11 75.0>, <B9 260.0>, <C12 0.0>, <B15 289.0>, <F12 76.0>, <F9 102.0>, <B10 228.0>, <D15 10.0>, <H12 30.0>, <J11 53.0>, <I9 7.0>, <I6 8.0>, <G16 774.0>, <G8 71.0>, <C14 1.0>, <F8 92.0>, <E15 31.0>, <B12 206.0>, <B13 219.0>, <E5 40.0>, <F13 76.0>, <J9 52.5>, <G5 54.0>, <E11 51.0>, <J6 50.0>, <D6 9.0>, <H16 341.0>, <D14 6.0>, <D9 12.0>, <J12 54.0>, <H13 29.0>, <C15 1.0>, <J15 55.0>, <D5 5.0>, <C6 1.0>, <D12 7.0>, <G10 62.0>, <D13 4.0>, <H6 25.0>, <C8 0.0>, <I12 2.0>, <I13 9.0>, <H14 40.0>, <I7 7.0>, <F6 77.0>, <E12 27.0>, <E10 52.0>, <F16 939.0>, <C9 0.0>, <I16 89.0>, <F5 78.0>, <G6 56.0>, <B7 243.0>, <I11 5.0>, <J14 54.0>, <G14 79.0>, <E6 62.0>, <H11 30.0>, <F15 90.0>, <H8 27.0>, <E7 33.0>, <G7 69.0>, <J13 55.0>, <D11 5.0>, <B14 282.0>, <F11 77.0>, <I15 15.0>, <C11 1.0>, <J5 52.0>, <B6 238.0>, <E8 43.0>, <C16 5.0>, <F10 73.0>, <D7 6.0>, <C10 0.0>, <B16 2668.0>, <F14 104.0>, <I10 8.0>, <D8 8.0>, <F7 94.0>, <G15 95.0>

In [6]:
age = tab.excel_ref('B4').expand(RIGHT).is_not_blank()
age

{<D4 '25-34'>, <G4 '55-64'>, <E4 '35-44'>, <C4 'Under 25'>, <F4 '45-54'>, <H4 '65-74'>, <I4 '75 and over'>}

In [7]:
age1 = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
age1

{<C3 'Age Group'>, <J3 'Median Age'>, <B3 'All Ages'>}

In [8]:
Year = tab.excel_ref('A5').expand(DOWN) - tab.excel_ref('A17').expand(DOWN)  
Year

{<A10 2011.0>, <A16 'Total (2006-2016)'>, <A6 2007.0>, <A13 2014.0>, <A8 2009.0>, <A12 2013.0>, <A7 2008.0>, <A11 2012.0>, <A15 2016.0>, <A14 2015.0>, <A9 2010.0>, <A5 2006.0>}

In [9]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,LEFT),
            HDim(age, 'Age',DIRECTLY,ABOVE),
            HDim(age1, 'Age1', DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,Year,Age,Age1,Measure Type,Unit
0,210.0,2006.0,None,All Ages,Count,People
1,1.0,2006.0,Under 25,Age Group,Count,People
2,5.0,2006.0,25-34,None,Count,People
3,40.0,2006.0,35-44,None,Count,People
4,78.0,2006.0,45-54,None,Count,People
5,54.0,2006.0,55-64,None,Count,People
6,22.0,2006.0,65-74,None,Count,People
7,10.0,2006.0,75 and over,None,Count,People
8,52.0,2006.0,None,Median Age,Count,People
9,238.0,2007.0,None,All Ages,Count,People


In [12]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [13]:
new_table['Year'] = new_table['Year'].astype(int)

In [14]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [15]:
new_table['Year'] = new_table['Year'].astype(str)

In [16]:
new_table['Value'] = new_table['Value'].astype(int)

In [17]:
new_table.dtypes

Value            int32
Year            object
Age             object
Age1            object
Measure Type    object
Unit            object
dtype: object

In [18]:
new_table.tail(5)

,Value,Year,Age,Age1,Measure Type,Unit
103,939,0,45-54,None,Count,People
104,774,0,55-64,None,Count,People
105,341,0,65-74,None,Count,People
106,89,0,75 and over,None,Count,People
107,54,0,None,Median Age,Count,People


In [19]:
def user_perc(x):
    
    if str(x) == '0':
        return '2001-2016'
    else:
        return x
    
new_table['Year'] = new_table.apply(lambda row: user_perc(row['Year']), axis = 1)


In [20]:
def user_perc1(x,y):    
    if str(x) == 'Median Age':
        return 'Median Age'
    else:
        return y        
    
new_table['Age'] = new_table.apply(lambda row: user_perc1(row['Age1'], row['Age']), axis = 1)

In [21]:
def user_perc2(x,y):    
    if str(x) == 'All Ages':
        return 'All Ages'
    else:
        return y        
    
new_table['Age'] = new_table.apply(lambda row: user_perc2(row['Age1'], row['Age']), axis = 1)

In [22]:
new_table

,Value,Year,Age,Age1,Measure Type,Unit
0,210,2006,All Ages,All Ages,Count,People
1,1,2006,Under 25,Age Group,Count,People
2,5,2006,25-34,None,Count,People
3,40,2006,35-44,None,Count,People
4,78,2006,45-54,None,Count,People
5,54,2006,55-64,None,Count,People
6,22,2006,65-74,None,Count,People
7,10,2006,75 and over,None,Count,People
8,52,2006,Median Age,Median Age,Count,People
9,238,2007,All Ages,All Ages,Count,People


In [23]:
new_table = new_table[['Year','Age','Measure Type','Value','Unit']]

In [24]:
new_table.head(5)

,Year,Age,Measure Type,Value,Unit
0,2006,All Ages,Count,210,People
1,2006,Under 25,Count,1,People
2,2006,25-34,Count,5,People
3,2006,35-44,Count,40,People
4,2006,45-54,Count,78,People


In [25]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('tab2a.csv'), index = False)